In [1]:
import pandas as pd

In [2]:
def transform(df):
    df.mask(df == "", inplace=True)
    df.drop(columns=['preciptype', 'severerisk', 'sunrise', 'sunset', 'moonphase', 'description', 'icon', 'stations'], inplace=True)
    df.loc[df["windgust"].isna(),'windgust'] = df['windspeed'] # if indgust is missing, insert windspeed
    df[['snow', 'snowdepth']] = df[['snow', 'snowdepth']].fillna(0)
    if "sealevelpressure" or "datetime" in df.columns:
        df.rename(columns={"sealevelpressure":"pressure", "datetime":"date"}, inplace=True)
    df['pressure'].fillna(df['pressure'].mean(), inplace=True)
    return df

def assemble(path_to_file):
    df = pd.read_csv(path_to_file, delimiter=',')
    new_df = transform(df)
    return new_df

In [4]:
final_df = assemble("C://Users//Admin//Desktop//github//ID2223//project//vienna_2013_2014.csv")

In [5]:
final_df = pd.concat([final_df,
                    assemble("C://Users//Admin//Desktop//github//ID2223//project//vienna_2014_2016.csv"),
                    assemble("C://Users//Admin//Desktop//github//ID2223//project//vienna_2016_2018.csv"),
                    assemble("C://Users//Admin//Desktop//github//ID2223//project//vienna_2018_2020.csv"),
                    assemble("C://Users//Admin//Desktop//github//ID2223//project//vienna_2020_2022.csv"),
                    assemble("C://Users//Admin//Desktop//github//ID2223//project//vienna_2022_2022.csv"),
                    assemble("C://Users//Admin//Desktop//github//ID2223//project//vienna_2022_2023.csv")])

In [8]:
import hopsworks
import os

os.environ['CONDA_DLL_SEARCH_MODIFICATION_ENABLE'] = '1'
client= hopsworks.login()
stream = client.get_feature_store()

weather_dt = stream.get_or_create_feature_group(
    name = 'weather_fg',
    description = 'Weather characteristics of each day',
    version=5,
    primary_key=['date'],
    online_enabled=True
)
weather_dt.insert(final_df)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5267


Connected. Call `.close()` to terminate connection gracefully.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5267/fs/5187/fg/15837


Uploading Dataframe: 0.00% |          | Rows 0/3302 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5267/jobs/named/weather_fg_5_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x299cde57c10>, None)

## Historical weather data EDA summary

- with my current subscription plan, I can only get 1000 days worth of data in every 24 hours
- 33 different parameters
- I dropped these variables: preciptype, severerisk, sunrise, sunset, moonphase, description, icon, stations.
- no missing dates, but lots of missing values consistently in windgust
- the way to deal with windgust: if missing, set it to the windspeed
- --------||---------- snow and snowdepth: set missing to zero

- 2020-2022: 1 nan in pressure
- 2014-2016: all snow missing, snowdepth 30 not nan
- 2013-2014: all snow missing, snowdepth 6 not nan
